<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

### Question 1

In [2]:
import requests # library to handle requests
import lxml.html as lh
import bs4 as bs
import urllib.request

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

In [3]:
## Source URL
url   = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [4]:
def scrape_table_bs4(cname,cols):
    page  = urllib.request.urlopen(url).read()
    soup  = bs.BeautifulSoup(page,'lxml')
    table = soup.find("table",class_=cname)
    header = [head.findAll(text=True)[0].strip() for head in table.find_all("th")]
    data   = [[td.findAll(text=True)[0].strip() for td in tr.find_all("td")]
              for tr in table.find_all("tr")]
    data    = [row for row in data if len(row) == cols]
    # Store data to this temporary dataframe
    raw_df = pd.DataFrame(data,columns=header)
    return raw_df
def scrape_table_lxml(XPATH,cols):
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    table_content = doc.xpath(XPATH)
    for table in table_content:
        headers = [th.text_content().strip() for th in table.xpath('//th')]
        headers = headers[0:3]
        data    = [[td.text_content().strip() for td in tr.xpath('td')] 
                   for tr in table.xpath('//tbody/tr')]
        data    = [row for row in data if len(row) == cols]
        raw_df = pd.DataFrame(data,columns=headers)
        return raw_df


# Data Cleanup
1. Drop/ignore cells with un-assigned boroughs.
2. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
3. Group the table by PostalCode/Borough, Neighbourhood belonging to same borough will be combined in 'Neighbourhood' column as separated with 'comma'.

In [9]:
Toronto_Postal = scrape_table_bs4("wikitable",3)
print(Toronto_Postal)

    Postcode           Borough  \
0        M1A      Not assigned   
1        M2A      Not assigned   
2        M3A        North York   
3        M4A        North York   
4        M5A  Downtown Toronto   
5        M6A        North York   
6        M6A        North York   
7        M7A  Downtown Toronto   
8        M8A      Not assigned   
9        M9A      Queen's Park   
10       M1B       Scarborough   
11       M1B       Scarborough   
12       M2B      Not assigned   
13       M3B        North York   
14       M4B         East York   
15       M4B         East York   
16       M5B  Downtown Toronto   
17       M5B  Downtown Toronto   
18       M6B        North York   
19       M7B      Not assigned   
20       M8B      Not assigned   
21       M9B         Etobicoke   
22       M9B         Etobicoke   
23       M9B         Etobicoke   
24       M9B         Etobicoke   
25       M9B         Etobicoke   
26       M1C       Scarborough   
27       M1C       Scarborough   
28       M1C  

In [10]:
# Only process the cells that have an assigned borough.  Ignore cells with a borough that is Not assigned.
Toronto_Postal_Codes=Toronto_Postal[~Toronto_Postal['Borough'].isin(['Not assigned'])]
Toronto_Postal_Codes=Toronto_Postal_Codes.sort_values(by=['Postcode','Borough','Neighbourhood'], ascending=[1,1,1]).reset_index(drop=True)
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. For example the value of the Borough and the Neighborhood columns will be Queen's Park.
Toronto_Postal_Codes.loc[Toronto_Postal_Codes['Neighbourhood'] == 'Not assigned', ['Neighbourhood']] = Toronto_Postal_Codes['Borough']
sample = Toronto_Postal_Codes.loc[Toronto_Postal_Codes['Borough'] == 'Queen\'s Park']
Toronto_Postal_Codes = Toronto_Postal_Codes.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [11]:
Toronto_Postal_Codes.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [8]:
TorontoPostalCodes.shape

(103, 3)

In [16]:
Toronto_Postal_Codes.to_csv('Toronto.TASK_1_df.csv',index=False)